# Loading and Exploring Data

In [7]:
# Importing necessary libraries
import pandas as pd
import numpy as np

# Loading the dataset
df = pd.read_csv('online_retail.csv', encoding='ISO-8859-1')

# Displaying the first few rows of the dataframe
print(df.head())

# Displaying basic information about the dataframe
print(df.info())

# Displaying summary statistics
print(df.describe())


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2  2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       ------------

# Data Pre-processing

In [8]:
# Checking for missing values
print(df.isnull().sum())

# Dropping rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])

# Filling missing values in Description with a placeholder
df['Description'] = df['Description'].fillna('No Description')

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [9]:
# Converting InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Converting CustomerID to string
df['CustomerID'] = df['CustomerID'].astype(str)

In [10]:
# Sorting data by CustomerID and InvoiceDate
df = df.sort_values(['CustomerID', 'InvoiceDate'])

# Creatng a function to calculate customer journey
def calculate_customer_journey(df):
    df['TransactionNo'] = df.groupby('CustomerID').cumcount() + 1
    df['TotalSpent'] = df['Quantity'] * df['UnitPrice']
    return df

# Applying the function to the dataframe
df = calculate_customer_journey(df)

# Displaying the updated dataframe
print(df.head())

      InvoiceNo StockCode                        Description  Quantity  \
61619    541431     23166     MEDIUM CERAMIC TOP STORAGE JAR     74215   
61624   C541433     23166     MEDIUM CERAMIC TOP STORAGE JAR    -74215   
14938    537626     85116    BLACK CANDELABRA T-LIGHT HOLDER        12   
14939    537626     22375  AIRLINE BAG VINTAGE JET SET BROWN         4   
14940    537626     71477  COLOUR GLASS. STAR T-LIGHT HOLDER        12   

              InvoiceDate  UnitPrice CustomerID         Country  \
61619 2011-01-18 10:01:00       1.04    12346.0  United Kingdom   
61624 2011-01-18 10:17:00       1.04    12346.0  United Kingdom   
14938 2010-12-07 14:57:00       2.10    12347.0         Iceland   
14939 2010-12-07 14:57:00       4.25    12347.0         Iceland   
14940 2010-12-07 14:57:00       3.25    12347.0         Iceland   

       TransactionNo  TotalSpent  
61619              1     77183.6  
61624              2    -77183.6  
14938              1        25.2  
14939       

In [11]:
# Checking for negative quantities and prices
print(df[(df['Quantity'] < 0) | (df['UnitPrice'] < 0)])

# Removing rows with negative quantities or prices
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Checking for duplicates
print(df.duplicated().sum())

# Removing duplicates
df = df.drop_duplicates()

# Verifying no missing values remain
print(df.isnull().sum())

       InvoiceNo StockCode                         Description  Quantity  \
61624    C541433     23166      MEDIUM CERAMIC TOP STORAGE JAR    -74215   
106394   C545329         M                              Manual        -1   
106395   C545329         M                              Manual        -1   
106397   C545330         M                              Manual        -1   
129737   C547388     84050     PINK HEART SHAPE EGG FRYING PAN       -12   
...          ...       ...                                 ...       ...   
488525   C577832     21108  FAIRY CAKE FLANNEL ASSORTED COLOUR       -18   
481908   C577386     23401       RUSTIC MIRROR WITH LACE HEART        -1   
481921   C577390     23401       RUSTIC MIRROR WITH LACE HEART        -1   
70604    C542086     22423            REGENCY CAKESTAND 3 TIER        -1   
295686   C562808     23187    FRENCH STYLE STORAGE JAR BONBONS        -5   

               InvoiceDate  UnitPrice CustomerID         Country  \
61624  2011-01-18 1

In [12]:
# Calculating RFM (Recency, Frequency, Monetary) metrics
import datetime as dt

# Defining snapshot date
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

# Aggregating data to get RFM metrics
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSpent': 'sum'
}).reset_index()

# Renaming columns
rfm.rename(columns={'InvoiceDate': 'Recency',
                    'InvoiceNo': 'Frequency',
                    'TotalSpent': 'Monetary'}, inplace=True)

# Displaying RFM metrics
print(rfm.head())

  CustomerID  Recency  Frequency  Monetary
0    12346.0      326          1  77183.60
1    12347.0        2        182   4310.00
2    12348.0       75         31   1797.24
3    12349.0       19         73   1757.55
4    12350.0      310         17    334.40


In [15]:
import plotly.express as px
import cufflinks as cf

# Enabling offline mode for cufflinks
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Plotting Distribution of Recency, Frequency, and Monetary metrics
fig = px.histogram(rfm, x='Recency', nbins=50, title='Recency Distribution')
fig.show()

fig = px.histogram(rfm, x='Frequency', nbins=50, title='Frequency Distribution')
fig.show()

fig = px.histogram(rfm, x='Monetary', nbins=50, title='Monetary Distribution')
fig.show()

# Scatter plot of RFM values
fig = px.scatter(rfm, x='Recency', y='Frequency', size='Monetary', color='Monetary',
                 title='RFM Segmentation', labels={'Recency': 'Recency (days)',
                                                   'Frequency': 'Frequency (transactions)',
                                                   'Monetary': 'Monetary (value)'})
fig.show()


These visualizations provide insights into customer behavior and spending patterns, essential for making informed business decisions.

### Recency Distribution
The Recency distribution histogram shows the number of days since the last purchase for each customer. The distribution indicates that most customers have made a purchase recently, with a significant number of customers showing high recency values. This suggests a healthy level of recent engagement with the business, although there is a long tail of customers who have not made recent purchases, which may indicate areas for re-engagement strategies.

### Frequency Distribution
The Frequency distribution histogram reveals the number of transactions made by each customer. The majority of customers have made only a few purchases, with a sharp decline in the number of customers as the frequency increases. This highlights the importance of identifying and nurturing high-frequency customers while implementing strategies to increase purchase frequency among the majority.

### Monetary Distribution
The Monetary distribution histogram illustrates the total monetary value spent by each customer. Similar to the frequency distribution, the monetary values are highly skewed, with most customers contributing relatively low amounts and a few high-value customers contributing significantly more. This emphasizes the need to focus on high-value customers and develop personalized marketing strategies to maximize their lifetime value.

### RFM Segmentation
The RFM segmentation scatter plot provides a comprehensive view of the relationship between recency, frequency, and monetary values. Customers are color-coded based on their monetary value, with high-value customers depicted in brighter colors. The plot shows a concentration of high-frequency, low-recency customers, indicating recent high engagement and spending. However, the spread of data also highlights segments of customers with varied engagement levels, pointing to opportunities for targeted interventions.

### Business Implications
**Customer Retention and Reactivation:** The high recency values for many customers suggest effective recent engagement, but the presence of customers with low recency indicates potential for reactivation campaigns. <br>
**Increasing Purchase Frequency:** Strategies to encourage more frequent purchases can help shift more customers into higher frequency categories, boosting overall sales. <br>
**Maximizing Customer Value:** High-value customers should be identified and given personalized attention to enhance their experience and increase their lifetime value. <br>
**Targeted Marketing:** The insights from RFM segmentation enable the creation of tailored marketing campaigns for different customer segments, optimizing marketing spend and improving ROI. <br>
By leveraging these insights, businesses can enhance their understanding of customer behavior, leading to more effective customer relationship management and improved business outcomes.